In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.python.client import device_lib


2024-10-01 19:33:29.099082: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 19:33:29.661879: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-01 19:33:29.737991: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-01 19:33:30.430536: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-01 19:33:33.522732: W tensorflow/compiler/tf2

In [2]:
# Check if TensorFlow is using a GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow is using the following GPU(s):")
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU detected for TensorFlow.")


TensorFlow is using the following GPU(s):
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
import tensorflow as tf
print(tf.__version__)


2.17.0


In [6]:
import time

# Create a random tensor
matrix1 = tf.random.normal([10000, 10000])
matrix2 = tf.random.normal([10000, 10000])

# Perform matrix multiplication on the GPU
start_time = time.time()
result = tf.matmul(matrix1, matrix2)
print("GPU computation time:", time.time() - start_time)

# Check if the operation was performed on the GPU
print("GPU used:", result.device)


2024-10-01 19:35:12.016575: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78852 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:41:00.0, compute capability: 8.0


GPU computation time: 4.285232782363892
GPU used: /job:localhost/replica:0/task:0/device:GPU:0


In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Allow TensorFlow to only use a limited amount of memory on the GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [7]:
# List all available devices
devices = device_lib.list_local_devices()
print("Available devices:")
for device in devices:
    print(device.name, device.device_type)


Available devices:
/device:CPU:0 CPU
/device:GPU:0 GPU


2024-10-01 19:36:06.847644: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /device:GPU:0 with 78852 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:41:00.0, compute capability: 8.0


In [8]:
# Paths
data_dir = 'data/training'  # Folder with .tif images
csv_file = 'data/training.csv'  # CSV file with image_id and is_homogeneous


In [9]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

In [10]:
df = pd.read_csv(csv_file)

In [11]:
# Strip any leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Now you can access 'image_id' without the extra space
image_id_values = df['image_id'].values
print(image_id_values)


[279 277 275 273 271 269 267 265 263 261 259 257 255 253 251 249 247 245
 243 241 239 237 235 233 231 229 227 225 223 221 219 217 215 213 211 209
 207 205 203 201 199 197 195 193 191 189 187 185 183 181 179 177 175 173
 171 169 167 165 163 161 159 157 155 153 151 149 147 145 143 141 139 137
 135 133 131 129 127 125 123 121 119 117 115 113 111 109 107 105 103 101
  99  97  95  93  91  89  87  85  83  81  79  77  75  73  71  69  67  65
  63  61  59  57  55  53  51  49  47  45  43  41  39  37  35  33  29  27
  25  23  21  19  17  15  13  11   9   7   5   3   1]


In [12]:
def load_and_preprocess_image(image_path):
    # Load image with PIL and convert to an array
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)
    # Normalize image pixel values (0-255 -> 0-1)
    img_array = img_array / 255.0
    return img_array

In [13]:
print(df.columns)



Index(['image_id', 'is_homogenous'], dtype='object')


In [14]:
# 3. Create lists of image paths and labels
# Assuming image_id values need to be 3 digits with leading zeros
image_paths = [os.path.join(data_dir, f"{str(image_id).zfill(3)}.tif") for image_id in df['image_id']]
labels = df['is_homogenous'].values

In [15]:
image_paths

['data/training/279.tif',
 'data/training/277.tif',
 'data/training/275.tif',
 'data/training/273.tif',
 'data/training/271.tif',
 'data/training/269.tif',
 'data/training/267.tif',
 'data/training/265.tif',
 'data/training/263.tif',
 'data/training/261.tif',
 'data/training/259.tif',
 'data/training/257.tif',
 'data/training/255.tif',
 'data/training/253.tif',
 'data/training/251.tif',
 'data/training/249.tif',
 'data/training/247.tif',
 'data/training/245.tif',
 'data/training/243.tif',
 'data/training/241.tif',
 'data/training/239.tif',
 'data/training/237.tif',
 'data/training/235.tif',
 'data/training/233.tif',
 'data/training/231.tif',
 'data/training/229.tif',
 'data/training/227.tif',
 'data/training/225.tif',
 'data/training/223.tif',
 'data/training/221.tif',
 'data/training/219.tif',
 'data/training/217.tif',
 'data/training/215.tif',
 'data/training/213.tif',
 'data/training/211.tif',
 'data/training/209.tif',
 'data/training/207.tif',
 'data/training/205.tif',
 'data/train

In [16]:
# 4. Load images and preprocess them
images = np.array([load_and_preprocess_image(image_path) for image_path in image_paths])



In [17]:
# 5. Split the data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


In [18]:
# 6. Create data generators for augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [19]:
val_datagen = ImageDataGenerator()


In [20]:
# 7. Create the data generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)
val_generator = val_datagen.flow(X_val, y_val, batch_size=BATCH_SIZE)


In [21]:
# 8. Load the pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [22]:
for layer in base_model.layers:
    layer.trainable = False

In [23]:
# 10. Create the model by adding custom layers on top of the pre-trained base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Add dropout for regularization
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

In [24]:
# 11. Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
# 12. Train the model
history = model.fit(
    train_generator,
    epochs=10,  # Adjust the number of epochs as needed
    validation_data=val_generator
)

Epoch 1/10


/home/cogu/.conda/envs/cellclass/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1727804174.881365   34976 service.cc:146] XLA service 0x7f3dc800be90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727804174.881409   34976 service.cc:154]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-10-01 19:36:14.917308: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-01 19:36:16.376219: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-10

7/7 ━━━━━━━━━━━━━━━━━━━━ 38s 3s/step - accuracy: 0.3430 - loss: 0.9428 - val_accuracy: 0.1429 - val_loss: 0.8891
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.3124 - loss: 0.9086 - val_accuracy: 0.1429 - val_loss: 0.7784
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.4836 - loss: 0.8145 - val_accuracy: 0.8571 - val_loss: 0.6902
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5785 - loss: 0.6507 - val_accuracy: 0.8571 - val_loss: 0.6198
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.6092 - loss: 0.6490 - val_accuracy: 0.8571 - val_loss: 0.5652
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.6730 - loss: 0.6302 - val_accuracy: 0.8571 - val_loss: 0.5254
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7230 - loss: 0.6297 - val_accuracy: 0.8571 - val_loss: 0.4960
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7814 - loss: 0.5109 - val_accuracy: 0.8571 - val_loss: 0.4719
Epoch 9/10
7

In [26]:
# 13. Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc*100:.2f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8423 - loss: 0.4608
Validation Accuracy: 85.71%


In [27]:
model.save('vgg16_homogeneous_classification.h5')
